In [1]:
import pandas as pd

from ApiHandler import ApiHandler

In [2]:
class ApiIndustryIndex(ApiHandler):
    settings = {
        **ApiHandler.settings,
        'ind_map': {
            'manufacturing': 1,
            'researching_time_efficiency': 3,
            'researching_material_efficiency': 4,
            'copying': 5,
            'invention': 8,
            'reaction': 11
        }
    }
    url = {
        **ApiHandler.url,
        'data': 'https://esi.evetech.net/latest/industry/systems/'
    }
    script_vals = {
        **ApiHandler.script_vals,
        'table': 'IndustrySysIndex',
        'data_cols': ['record_time', 'solar_system_id', 'activity_id', 'cost_index'],
        'table_cols': ['record_time', 'system_id', 'activity_id', 'cost_index']
    }
    name = 'ESI Industry Index API'
    
    def build_data(self, raw_data):
        if self.verbose: self._verbose('build_data', 'Building data frame...')
        
        data_frame = []
        for rcd in js.loads(raw_data):
            for act in rcd['cost_indices']:
                data_frame.append({
                    'record_time': self.timestamp,
                    'solar_system_id': rcd['solar_system_id'],
                    'activity_id': self.settings['ind_map'][act['activity']],
                    'cost_index': act['cost_index']
                })
        data_frame = pd.DataFrame(data_frame)
        
        if self.verbose: self._verbose('build_data', 'Data frame built. %s records.' % len(data_frame))
            
        return data_frame
    
    self.data_inserter(self, data_frame):
        if self.verbose: self._verbose('insert_data', 'Inserting records...')
        
        insert_script = self.sql['insert'].format(**{
            'table':self.script_vals['table']
            'cols':','.join(self.script_vals['table_cols']),
            'vals':','.join(['%s']*len(self.script_vals['table_cols']))
        })
        
        cur = self.conn.cursor()
        for row in data_frame[self.data_cols].itertuples():
            cur.execute(insert_script, list(row[1:]))
        self.conn.commit()
        cur.close()
        
        if self.verbose: self._verbose('insert_data', 'Records inserted.')

In [3]:
api = ApiIndustryIndex()

In [4]:
raw_data = api.get_raw_data()
data_frame = api.build_data(raw_data)

In [5]:
data_frame.head()

,activity_id,cost_index,record_time,system_id
0,1,0.0208,2019-08-03 19:41:51,30020141
1,3,0.0214,2019-08-03 19:41:51,30020141
2,4,0.0237,2019-08-03 19:41:51,30020141
3,5,0.0212,2019-08-03 19:41:51,30020141
4,8,0.0471,2019-08-03 19:41:51,30020141


In [6]:
api.connect_mdb()
test = pd.read_sql("""\
SELECT *
FROM IndustrySysIndex
LIMIT 10
;""", api.conn)
api.conn.close()
test.head()

,record_time,system_id,index_type,index_value
0,2019-01-09 09:45:18,30000001,1,0.0150
1,2019-01-09 09:45:18,30000001,3,0.0212
2,2019-01-09 09:45:18,30000001,4,0.0194
3,2019-01-09 09:45:18,30000001,5,0.0219
4,2019-01-09 09:45:18,30000001,8,0.0218
